# fmu.sumo.explorer

Package for browsing cases and surfaces in Sumo.

In [ ]:
from fmu.sumo.explorer import Explorer
import xtgeo
from io import BytesIO
%matplotlib inline

## Class: Explorer
Search for cases in SUMO

Methods:
- get_fields()
- get_users()
- get_status()
- get_cases()

In [ ]:
sumo = Explorer(env="dev", write_back=True)

### Explorer.get_fields()

Returns:

```
Dict
{
    FIELD_NAME: CASE_COUNT
}
```

In [ ]:
fields = sumo.get_fields()

print(fields)

### Explorer.get_users()

Returns:

```
Dict
{
    USER_NAME: CASE_COUNT
}
```

In [ ]:
users = sumo.get_users()

print(users)

### Explorer.get_status()

Returns:

`Dict`

```
{
    STATUS: CASE_COUNT
}
```

In [ ]:
status = sumo.get_status()

print(status)

### Explorer.get_cases()

Params:
- status: string[] 
- fields: string[]
- users: string[]

Returns: `CaseCollection`

In [ ]:
my_cases = sumo.get_cases(
    status=["keep", "official"],
    fields=["JOHAN SVERDRUP"],
    #users=["peesv", "jsorb"]
)

print(len(my_cases))

for case in my_cases:
    print(case.case_name)

### Explorer.get_case_from_id()

Params:
- sumo_id: string

Returns: `Case`

In [ ]:
my_case = sumo.get_case_by_id("81a57a32-37e7-06bc-924e-6710ba6e59b0")

print(my_case.case_name)

## Class: Case

Represents a case in Sumo and has methods for finding surfaces belonging to the case.

Some of the case meta data is accessible as attributes, the rest can be extracted from `Case.meta_data`.

Attributes:
- meta_data: dict
- sumo_id: uuid
- fmu_id: uuid
- case_name: string
- field_name: string
- status: string
- user: string

Methods:
- get_object_types()
- get_iterations()
- get_realizations()
- get_object_names()
- get_object_tag_names()
- get_object_aggregations()
- get_object_timespans()
- get_object()

Raw metadata object can be accessed at `Case.meta_data`

In [ ]:
# Some case info as attributes
print(my_case.sumo_id)
print(my_case.fmu_id)
print(my_case.case_name)
print(my_case.field_name)
print(my_case.status)
print(my_case.user)

### Case.get_object_types()

Returns:
```
Dict
{
    OBJECT_NAME: OBJECT_COUNT
}
```

In [ ]:
object_types = my_case.get_object_types()

print(object_types)

### Case.get_iterations()

Returns:
```
Dict
{
    ITERATION_ID: OBJECT_COUNT
}
```

In [ ]:
iterations = my_case.get_iterations()

print(iterations)

### Case.get_realizations()

Params:
- iteration_id: number

Returns:
```
Dict
{
    REALIZATION_ID: OBJECT_COUNT
}
```

In [ ]:
realizations = my_case.get_realizations(iteration_id=0)

print(realizations)

### Case.get_object_property_values()

Get distinct values for a property for a specific object type. I.e: get distinct object names for surfaces.
Can be used to get values that can be used in filtering when retrieving objects.

This method is mean to be dynamic and flexible and can replace all the other `get_object_*`-methods:
- get_object_tag_name
- get_object_names
- get_object_aggregations
- get_object_time_intervals


Params:
- property: string
- object_type: string
- object_name: string
- tag_name: string
- iteration_id: number
- realization_id: number
- aggregation: string

`property` and `object_type` are required, the rest of the parameters are used for filtering and are optional.

Valid `property` values:
- tag_name
- object_name
- time_interval
- aggregation
- iteration_id
- realization_id

Valid `object_type` values:
- surface
- polygons
- table

Returns 
```
Dict {
    [PROPERTY VALUE]: COUNT
}
```

#### Example: filter down to unique surface object

This example uses the `get_object_property_values` in several steps to get values that uniquley identifies a surface object.

In [ ]:
# get iteration_ids
iteration_ids = my_case.get_object_property_values(
    property="iteration_id",
    object_type="surface"
)

print(iteration_ids)

iteration_id = list(iteration_ids.keys())[0]

# use the retrieved iteration_id to get tag_names for surfaces within this iteration
tag_names = my_case.get_object_property_values(
    property="tag_name",
    object_type="surface",
    iteration_id=iteration_id,
)

print(tag_names)

tag_name = list(tag_names.keys())[5]

# use the retrieved tag_name to get object_names for surfaces containing this tag_name
object_names = my_case.get_object_property_values(
    property="object_name",
    object_type="surface",
    iteration_id=iteration_id,
    tag_name=tag_name
)

print(object_names)

object_name = list(object_names.keys())[2]

# use the object_name, tag_name and iteration_id to get available time_intervals
time_intervals = my_case.get_object_property_values(
    property="time_interval",
    object_type="surface",
    iteration_id=iteration_id,
    tag_name=tag_name,
    object_name=object_name
)

print(time_intervals)

time_interval = list(time_intervals.keys())[2]

# use the object_name, tag_name and iteration_id to get available aggregations
aggregations = my_case.get_object_property_values(
    property="aggregation",
    object_type="surface",
    iteration_id=iteration_id,
    tag_name=tag_name,
    object_name=object_name
)

print(aggregations)

aggregation = list(aggregations.keys())[3]

# get available realization_ids based on iteration_id, object_name, tag_name and time_interval
realization_ids = my_case.get_object_property_values(
    property="realization_id",
    object_type="surface",
    iteration_id=iteration_id,
    tag_name=tag_name,
    object_name=object_name,
    time_interval=time_interval
)

print(realization_ids)

realization_id = list(realization_ids.keys())[7]

# get surface from realization
surfaces = my_case.get_objects(
    object_type="surface",
    iteration_id=iteration_id,
    tag_name=tag_name,
    object_name=object_name,
    time_interval=time_interval,
    realization_id=realization_id
)

print(len(surfaces))

# get aggregated surface
surfaces = my_case.get_objects(
    object_type="surface",
    iteration_id=iteration_id,
    tag_name=tag_name,
    object_name=object_name,
    time_interval=time_interval,
    aggregation=aggregation
)

print(len(surfaces))

### Case.get_object_tag_names()

Params:
- object_type: string
- iteration_id: number
- realization_id: number
- aggregation: string

<strong>NOTE:</strong>
If no aggregation-parameter is provided, the search will exclude tag names from aggregated objects

Returns:

```
Dict
{
    TAG_NAME: COUNT
}
```


In [ ]:
tag_names = my_case.get_object_tag_names(
    object_type="surface",
    iteration_id=0, 
    realization_id=0,
    #aggregation="MEAN"
)

print(tag_names)

### Case.get_object_names()

Params:
- object_type: string
- tag_name: string
- iteration_id: number
- realization_id: number
- aggregation: string

<strong>NOTE:</strong>
If no aggregation-parameter is provided, the search will exclude surface names from aggregated objects

Returns:
```
Dict
{
    SURFACE_NAME: COUNT
}
```

In [ ]:
surface_names = my_case.get_object_names(
    object_type="surface",
    tag_name="amplitude_full_max",
    iteration_id=0, 
    realization_id=0,
    #aggregation="MEAN"
)

print(surface_names)

### Case.get_object_aggregations()

Params:
- object_type: string
 - object_name: string
 - tag_name: string
 - iteration_id: number

Returns:
````
Dict
{
    AGGREGATION_OPERATION: COUNT
}

In [ ]:
aggregations = my_case.get_object_aggregations(
    object_type="surface",
    object_name="draupne_fm_1",
    tag_name="amplitude_full_max",
    iteration_id=0,
)

print(aggregations)

### Case.get_object_times_intervals()

Params:
- object_type: string
- object_name: string
- tag_name: string
- iteration_id: number
- realization_id: number
- aggregation: string

Returns:
`````
Dict
{
    TIMESPAN: COUNT
}

In [ ]:
intervals = my_case.get_object_time_intervals(
    object_type="surface",
    object_name="draupne_fm_1",
    tag_name="amplitude_full_max",
    iteration_id=0,
    #realization_id=0,
    aggregation="MEAN"
)

print(intervals)

### Case.get_objects()

Params:
- object_type: string
- object_name: string
- tag_name: string
- iteration_id: number
- realization_id: number
- aggregation: string

Returns:
`DocumentCollection[ChildObject]`

In [ ]:
# Get a surface object from specific realization
surfaces = my_case.get_objects(
    object_type="surface",
    object_name="draupne_fm_1",
    tag_name="amplitude_full_max",
    time_interval="2019-10-01 - 2020-10-01",
    iteration_id=0,
    realization_id=0
)

s = surfaces[0]

print(s.name)
print(s.tag_name)

In [ ]:
# Get all versions of surface object (1 from each realization) for a specified iteration
surfaces = my_case.get_objects(
    object_type="surface",
    object_name="Aasgard Fm. Top",
    tag_name="structural_model",
    iteration_id=0
)

print(len(surfaces))

In [ ]:
# Get specific aggregated surface
surfaces = my_case.get_objects(
    object_type="surface",
    object_name="draupne_fm_1",
    tag_name="amplitude_full_max",
    time_interval="2019-10-01 - 2020-10-01",
    iteration_id=0,
    aggregation="MEAN"
)

s = surfaces[0]

print(s.name)
print(s.tag_name)

In [ ]:
# Visualize aggregated surface with xtgeo
bytestring = BytesIO(s.blob)
xtgeo_surface = xtgeo.surface_from_file(bytestring)
xtgeo_surface.quickplot()

In [ ]:
# Get surface as png + visualize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

bytestring = BytesIO(s.png)
img = mpimg.imread(bytestring)
imgplot = plt.imshow(img)
plt.show()

## Class: DocumentCollection

A set of documents from Sumo. Inherits from collections.Sequence and acts as a list of documents.

In [ ]:
surfaces = my_case.get_objects(
    object_type="surface",
    object_name="Aasgard Fm. Top",
    tag_name="structural_model",
    iteration_id=0
)

for s in surfaces[1:3]:
    print(s.name)
    print(s.realization_id)
    print(s.object_type)

### ObjectCollection.aggregate()

On-demand aggregation of objects. Currently only supports surface objects.

Params:
- operations: string || string[]

Returns: blob if one operation is provided, dictionary of blobs if multiple operations:
```
Dict
{
    OPERATION: BLOB
}
```

In [ ]:
surfaces = my_case.get_objects(
    object_type="surface",
    object_name="Aasgard Fm. Top",
    tag_name="structural_model",
    iteration_id=0
)

print(len(surfaces))

# One operation
mean = surfaces.aggregate("MEAN")

# Multiple
#aggs = surfaces.aggregate(["MEAN", "MIN", "MAX"])
#mean = aggs["MEAN"]

In [ ]:
# Visualize aggregated surface with xtgeo
bytestring = BytesIO(mean)
xtgeo_surface = xtgeo.surface_from_file(bytestring)
xtgeo_surface.quickplot()

## Class: ChildObject
Represents a child object in Sumo. 

Some of the meta data is accessible as attributes, the rest can be extracted from `ChildObject.meta_data`

Attributes:
- meta_data: dict
- sumo_id:uuid
- iteration_id: number
- realization_id: number
- name: string
- relative_path: string
- full_path: string
- aggregation: string
- object_type: string

Properties:
- blob: blob

In [ ]:
s = surfaces[0]

# Get meta_data and blob
meta_data = s.meta_data
blob = s.blob

# Get data from meta_data attribute
file_path = meta_data["file"]["relative_path"]

# Or use attribute
file_path = s.relative_path

## Example: process of finding a surface from case


In [ ]:
my_case = sumo.get_case_by_id("81a57a32-37e7-06bc-924e-6710ba6e59b0")

# Get iteration
iterations = my_case.get_iterations()

i = list(iterations.keys())[0]


# Get realization
realizations = my_case.get_realizations(
    iteration_id=i
)

r = list(realizations.keys())[0]


# Get tag names
tag_names = my_case.get_object_tag_names(
    object_type="surface",
    iteration_id=i,
    realization_id=r
)

t = list(tag_names.keys())[0]


# Get surface name
surface_names = my_case.get_object_names(
    object_type="surface",
    tag_name=t,
    iteration_id=i,
    realization_id=r
)

n = list(surface_names.keys())[0]


# Get surface
surfaces = my_case.get_objects(
    object_type="surface",
    object_name=n,
    tag_name=t,
    iteration_id=i,
    realization_id=r
)

s = surfaces[0]

# Visualize with xtgeo
bytestring = BytesIO(s.blob)
xtgeo_surface = xtgeo.surface_from_file(bytestring)
xtgeo_surface.quickplot()

# Print info
print(f"Case: {my_case.case_name}")
print(f"Iteration: {i}")
print(f"Realization: {r}")
print(f"Surface name: {n}")
print(f"Tag name: {t}")